<a href="https://colab.research.google.com/github/Taha-Sayyed/Machine-Learning-Project/blob/main/Extraction_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
!pip install mtcnn

In [97]:
import cv2
import os
from google.colab.patches import cv2_imshow
import numpy as np
from mtcnn import MTCNN


In [98]:
# Initialize the MTCNN face detector.
detector = MTCNN()

In [107]:
# --- Extraction Code ---
video_path = "/content/drive/My Drive/Face_Recognition/musfira.mp4"
vid = cv2.VideoCapture(video_path)
if not vid.isOpened():
    print("Error: Video file cannot be opened. Please check the file path and file format.")
else:
    print("Video file opened successfully.")

Video file opened successfully.


In [108]:
# Get the video frame rate and use a default if FPS is not available.
fps = vid.get(cv2.CAP_PROP_FPS)
if fps == 0:
    print("FPS not detected, using default FPS = 30")
    fps = 30


In [109]:
# Calculate sampling interval to extract 5 frames per second.
sample_rate = int(round(fps / 5))
print(f"Video FPS: {fps}, extracting every {sample_rate} frame(s) to get ~5 frames per second.")


Video FPS: 30.00119210034836, extracting every 6 frame(s) to get ~5 frames per second.


In [111]:
# Output directory for original and augmented images.
dir_path = "/content/drive/My Drive/Face_Recognition/Dataset/Musfira_Sayyed"
if not os.path.exists(dir_path):
    os.makedirs(dir_path)
    print(f"Creating the directory: {dir_path}")
else:
    print(f"{dir_path} already exists")

/content/drive/My Drive/Face_Recognition/Dataset/Musfira_Sayyed already exists


In [112]:
frame_counter = 0
extracted_frames = 0

In [113]:
# Augmentation parameters.
resize_dim = (160, 160)
h_shift = 20
v_shift = 20
brightness_alpha = 1.2
brightness_beta = 30
darkness_alpha = 0.7
darkness_beta = -30
blur_ksize = (7, 7)
padding = 20  # padding in pixels

In [ ]:
while True:
    success, frame = vid.read()
    if not success:
        break

    # Process only every sample_rate-th frame.
    if frame_counter % sample_rate == 0:
        # --- Padding the frame ---
        padded_frame = cv2.copyMakeBorder(frame, padding, padding, padding, padding, cv2.BORDER_CONSTANT, value=[0, 0, 0])

        # --- Face Extraction using MTCNN ---
        detections = detector.detect_faces(padded_frame)
        if len(detections) == 0:
            print(f"No face detected in frame {frame_counter}. Skipping this frame.")
        else:
            # Using the first detected face.
            detection = detections[0]
            x, y, w, h = detection['box']
            # Ensure coordinates are positive.
            x = max(0, x)
            y = max(0, y)
            face = padded_frame[y:y+h, x:x+w]

            # Resize the cropped face image.
            resized_face = cv2.resize(face, resize_dim)

            # --- Save the Original Resized Face ---
            original_filename = f"{dir_path}/frame_{extracted_frames}_original.jpg"
            cv2_imshow(resized_face)  # Display the resized face image in Colab.
            cv2.imwrite(original_filename, resized_face)

            # --- Augmentation Steps ---
            # 1. Horizontal Shift with Black Padding.
            h_shifted = np.roll(resized_face, h_shift, axis=1)
            if h_shift > 0:
                h_shifted[:, :h_shift] = 0
            else:
                h_shifted[:, h_shift:] = 0

            # 2. Vertical Shift with Black Padding.
            v_shifted = np.roll(resized_face, v_shift, axis=0)
            if v_shift > 0:
                v_shifted[:v_shift, :] = 0
            else:
                v_shifted[v_shift:, :] = 0

            # 3. Brightness Adjustment.
            bright = cv2.convertScaleAbs(resized_face, alpha=brightness_alpha, beta=brightness_beta)

            # 4. Darkness Adjustment.
            dark = cv2.convertScaleAbs(resized_face, alpha=darkness_alpha, beta=darkness_beta)

            # 5. Blurring.
            blur = cv2.GaussianBlur(resized_face, blur_ksize, 0)

            # 6. Flipping (Horizontal).
            flip = cv2.flip(resized_face, 1)

            # Save augmented images with descriptive filenames.
            cv2.imwrite(f"{dir_path}/frame_{extracted_frames}_hshift.jpg", h_shifted)
            cv2.imwrite(f"{dir_path}/frame_{extracted_frames}_vshift.jpg", v_shifted)
            cv2.imwrite(f"{dir_path}/frame_{extracted_frames}_bright.jpg", bright)
            cv2.imwrite(f"{dir_path}/frame_{extracted_frames}_dark.jpg", dark)
            cv2.imwrite(f"{dir_path}/frame_{extracted_frames}_blur.jpg", blur)
            cv2.imwrite(f"{dir_path}/frame_{extracted_frames}_flip.jpg", flip)

            extracted_frames += 1

    frame_counter += 1

vid.release()
print(f"Total extracted frames (and augmented sets): {extracted_frames}")


In [115]:
# List all files in the directory and count them.
all_files = os.listdir(dir_path)
# Optionally, filter to count only JPEG images (if your frames are saved as .jpg).
frame_files = [f for f in all_files if f.lower().endswith('.jpg')]
print("Total frames (including augmented images):", len(frame_files))

Total frames (including augmented images): 182
